<h3>Задача*</h3>

Есть сайт с графиками общественного транспорта [Kudikina](https://kudikina.ru). <br><br>
Спарсим в единую таблицу все маршруты автобусов Красноярска с графиками посещения остановок (с минимальными изменениями можно адаптировать для любого города или вида транспорта). <br><br>
Структура таблицы:
- Номер маршрута
- Тип маршрута (для красноярских автобусов это А и В — прямой и обратный)
- Описание (начальная и конечная остановка)
- День недели
- Остановка
- Время посещения

*Это реальная задача, требовавшая решения в одной из компаний, связанных с городскими исследованиями.


In [1]:
from bs4 import BeautifulSoup
import requests as req
import lxml
import re
import pandas as pd
import numpy as np

In [10]:
base_link = 'https://kudikina.ru'
routes_links = []
routes_numbers = req.get('https://kudikina.ru/krsk/bus/') # обозначим область поиска - автобусы Красноярска
routes_numbers = BeautifulSoup(routes_numbers.content, 'lxml')
for elem in routes_numbers.find('div', attrs={'text-center'}).find_all('a', href = True): # получим ссылки на все маршруты
  link = base_link + elem['href']
  page = req.get(link)
  soup = BeautifulSoup(page.content, 'lxml')
  for tag in soup.find_all('li'): # получим ссылки на подварианты маршрутов
    routes_links.append(base_link + tag.a.attrs['href'])

krasnoyarsk_bus = pd.DataFrame(columns=['route', 'way', 'description', 'weekday', 'stop', 'time'])

count = 0
for link in routes_links: # пройдём по части маршрутов
  page = req.get(link) # получаем страницу
  soup = BeautifulSoup(page.content, 'lxml')
  number = re.search(' .*,' ,soup.find('h3', attrs = {'text-center padding-15'}).text).group()[2:-1] # номер маршрута
  way = link[-1] # направление - прямое или обратное
  description = re.search(',.*',  soup.find('h3', attrs = {'text-center padding-15'}).text).group()[2:] # начальная и конечная остановки
  for day in range(1,8): # перебираем дни
    route_day = pd.DataFrame(columns=['route', 'way', 'description', 'weekday', 'stop', 'time']) # собирать данные лучше во вспомогательный датасет (ускорение ~ в 6-7 раз)
    day_link = link + '?d=' + str(day)
    page = req.get(day_link)
    soup = BeautifulSoup(page.content, 'lxml')
    if len(soup.find_all('div', attrs = {'alert alert-info'})) > 1: # есть дни, в которые маршрут не активен, обнаруживаем их по табличке оповещения (1 таблица - норма)
      continue
    else:
      stops = soup.find_all('div', attrs = {'row'})[3:-1] # отсекаем 4 элемента - везде несут иной функционал
      for elem in stops:
        stop = elem.a.text # название остановки
        times = elem.find_all('tr') # soup с таблицами времён этой остановки
        for time in times: # заходим в конкретный час
          hour = time.find_all('td')[0].text
          minutes = time.find_all('td')[1]
          for minute in minutes.find_all('span'): # перебираем конкретные минуты
            stop_time = hour + ':' + minute.text
            route_day.loc[len(route_day)] = number, way, description, day, elem.a.text, stop_time # заполняем данные
    krasnoyarsk_bus = pd.concat([krasnoyarsk_bus, route_day], ignore_index = True) # скидываем данные в общий датасет
  count += 1
  print('Progress: {}/{}'.format(count, len(routes_links)))

Progress: 1/131
Progress: 2/131
Progress: 3/131
Progress: 4/131
Progress: 5/131
Progress: 6/131
Progress: 7/131
Progress: 8/131
Progress: 9/131
Progress: 10/131
Progress: 11/131
Progress: 12/131
Progress: 13/131
Progress: 14/131
Progress: 15/131
Progress: 16/131
Progress: 17/131
Progress: 18/131
Progress: 19/131
Progress: 20/131
Progress: 21/131
Progress: 22/131
Progress: 23/131
Progress: 24/131
Progress: 25/131
Progress: 26/131
Progress: 27/131
Progress: 28/131
Progress: 29/131
Progress: 30/131
Progress: 31/131
Progress: 32/131
Progress: 33/131
Progress: 34/131
Progress: 35/131
Progress: 36/131
Progress: 37/131
Progress: 38/131
Progress: 39/131
Progress: 40/131
Progress: 41/131
Progress: 42/131
Progress: 43/131
Progress: 44/131
Progress: 45/131
Progress: 46/131
Progress: 47/131
Progress: 48/131
Progress: 49/131
Progress: 50/131
Progress: 51/131
Progress: 52/131
Progress: 53/131
Progress: 54/131
Progress: 55/131
Progress: 56/131
Progress: 57/131
Progress: 58/131
Progress: 59/131
Progre

In [11]:
krasnoyarsk_bus

,route,way,description,weekday,stop,time
0,СФУ,A,Сибирский Федеральный Университет - Студенческ...,1,Сибирский федеральный университет,7:50
1,СФУ,A,Сибирский Федеральный Университет - Студенческ...,1,Сибирский федеральный университет,9:20
2,СФУ,A,Сибирский Федеральный Университет - Студенческ...,1,Сибирский федеральный университет,11:20
3,СФУ,A,Сибирский Федеральный Университет - Студенческ...,1,Сибирский федеральный университет,13:30
4,СФУ,A,Сибирский Федеральный Университет - Студенческ...,1,Сибирский федеральный университет,15:10
...,...,...,...,...,...,...
2356565,99,B,Автотранспортный техникум - мкрн. Северный,7,станция Красноярск-Северный,22:46
2356566,99,B,Автотранспортный техникум - мкрн. Северный,7,станция Красноярск-Северный,23:03
2356567,99,B,Автотранспортный техникум - мкрн. Северный,7,станция Красноярск-Северный,23:21
2356568,99,B,Автотранспортный техникум - мкрн. Северный,7,станция Красноярск-Северный,23:38


Скрипт можно легко адаптировать под функцию, в которой указывался бы маршрут, и день. Или, расширяя географию, город и тип транспорта (сайт однороден, скрипт можно масштабировать). <br><br>
Если мы заинтересованы работать именно, например, с автобусами Красноярска, то более предпочтительным кажется вариант полного парсинга и обращения с базе данных. <br><br>
Если же нас могут интересовать разные города, разные виды транспорта и разные конкретные маршруты, то, вполне возможно, проще именно несколько доработать скрипт и использовать его в функции, парся 1 конкретный маршрут.
